In [2]:
import torch
import torch.nn as nn

import torch.optim as optim
import torchvision 
import torchvision.transforms as transforms

from torch.utils.data import DataLoader
from tqdm import tqdm

In [4]:
# 0. 장치설정
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using device : {device}')

Using device : cuda


In [7]:
# 1. 데이터준비
# 1.1 데이터 전처리
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5,),(0.5,))
    
    
    
])

# 1.2 데이터 다운로드, 로딩
training_set = torchvision.datasets.FashionMNIST( #데이터셋은 이미 만들어져옴(근데 회사에서는 dataset직접 만들어야함,)
    './data',
    train=True,
    download=True, # 없으면 다운로드 행라 없으면 하지말라
    transform=transform
)

# 1.3 데이터로더 생성
training_loader = DataLoader(
    dataset = training_set,
    batch_size = 64,
    shuffle=True # 한개 이미지가 옆에 다른이미지와 상관없어서 shuffle 상관 X 
)



In [9]:
# 2.모델 클래스 정의
class MLP(nn.Module):
    def __init__(self) :
        super(MLP,self).__init__()
        self.flatten = nn.Flatten()

        self.layers = nn.Sequential(
            nn.Linear(784,512),
            nn.ReLU(),
            nn.Linear(512,256),
            nn.ReLU(),
            nn.Linear(256,10)
        )

    def forward(self,x): # x:입력데이터
        x = self.flatten(x)
        x = self.layers(x)
        return x 

In [11]:
# 2.1 모델 생성,모델을 cuda 즉, gpu로 이동하겠다.
# model = MLP() # MLP object 생성 
model = MLP().to(device) # cuda 메모리에 보내겠다.

# 2.2 손실함수,옵티마이저 생성
loss_fn = nn.CrossEntropyLoss() # 오차 구하는 함수
optimizer = optim.Adam(model.parameters(), lr=0.0001) # lr 은 적당히로 해야하는데 적당히를 모름
# lr 이 작을수록 천천히 내려감, 시간 많이 걸리지만,발산확률이 젤 작음

In [13]:
# 3.검증용 데이터 다운로드, 데이터로더 생성
validation_set = torchvision.datasets.FashionMNIST(
    './data',
    train=False,
    download=True,
    transform=transform
)

validation_loader = DataLoader(
    dataset=validation_set,
    batch_size=64,
    shuffle=False # 검증용데이터는 shuffle X --> False
)

In [ ]:
# 20251226

In [ ]:
# 4.훈련 및 검증 루프
num_epochs = 5
for epoch in range(num_epochs):
    # 여기서부터 훈련
    model.train() # 모델을 훈련 모드로 설정
    running_loss = 0.0
    progress_bar = tqdm(training_loader,desc=f'Epoch {epoch+1} [Train]')
    
    for dataset in progress_bar: #6만/64 횟수만큼 반복
        # 학습에 필요한 데이터(배치) 준비
        input_data,labels = dataset
        input_data, labels = input_data.to(device),labels.to(device)

        # 진짜 여기서부터 학습 시작
        # 실제 학습은 optimizer가 함
        optimizer.zero_grad()
        outputs= model(input_data)
        loss=loss_fn(outputs, labels)
        loss.backward()
        optimizer.step() # backward,step이 반복됨

        # 여기까지 1 batch 데이터의 학습 종료
        running_loss += loss.item()
        progress_bar.set_postfix({'train_loss' : running_loss/(progress_bar.n+1)}) # n:횟수
    
    # 에폭별 평균 오차 출력
    avg_train_loss =running_loss/len(training_loader)

    # --- 검증 루프---
    running_vloss = 0.0 # 초기값
    correct = 0
    total = 0

    # 검증 모드로 변경
    # <주의> 검증모드는 순전파만 해야함(즉, 결과값만 뽑아내야함), 역전파를 막아야함!!

    model.eval()

    # 기울기 계산을 비활성화
    with torch.no_grad(): # 역전파 하지마라
        for vdataset in validation_loader: # 반복 => 1만/64
            vinputs, vlabels = vdataset
            vinputs, vlabels = vinputs.to(device), vlabels.to(device)

            # 순전파: 예측값 추출(64,10)
            voutputs = model(vinputs) # 결과가 (64,10)으로 나옴
            
            # 출력용 손실을 계산하겠다
            vloss= loss_fn(voutputs, vlabels)
            running_vloss += vloss.item() #손실을 누적(합) => 평균
            
            # 정확도 계산
            # 예측값 추출 0~9, 원래답: 0~9 --> 64개
            _,predicted =torch.max(input=voutputs,dim=1) 
            # 안쓸껀데 erro는 안나야하니까 _써야함
            # tensor니까 tensor 를 다루는 torch로 max 때려야함

            total += vlabels.size(0)
            # 갯수 누적
            # 64를 적는 하드코딩을 하면 안됨
            
            # 정확도 계산 누적한다.
            correct += (predicted==vlabels).sum().item() # 맞은갯수 추출(64개 가운데)
    #평균 손실
    avg_val_loss = running_vloss / len(validation_loader)
    
    #백분위로 출력
    accuracy = correct/total*100 

    # epoch for문 
    print(f'Epoch {epoch+1} Summary | Train Loss: {avg_train_loss:.4f} | Val Loss : {avg_val_loss:.4f} | Val Acc : {accuracy:.2f}%')

# end for Train finished
print('\n Finished Training!!! ')

    # <주의> 1epoc 가 끝나고 검증해야함, 한번에 검증 하는게 아님, 그래야 제대로 학습하는지 알 수 있음

 

Epoch 1 [Train]: 100%|██████████| 938/938 [00:22<00:00, 42.25it/s, train_loss=0.49] 


Epoch 1 Summary | Train Loss: 0.4885 | Val Loss : 0.4492 | Val Acc : 83.89%


Epoch 2 [Train]: 100%|██████████| 938/938 [00:21<00:00, 43.20it/s, train_loss=0.404]


Epoch 2 Summary | Train Loss: 0.4027 | Val Loss : 0.4127 | Val Acc : 84.85%


Epoch 3 [Train]: 100%|██████████| 938/938 [00:22<00:00, 42.10it/s, train_loss=0.369]


Epoch 3 Summary | Train Loss: 0.3680 | Val Loss : 0.3895 | Val Acc : 86.05%


Epoch 4 [Train]: 100%|██████████| 938/938 [00:21<00:00, 42.82it/s, train_loss=0.346]


Epoch 4 Summary | Train Loss: 0.3452 | Val Loss : 0.3822 | Val Acc : 86.09%


Epoch 5 [Train]: 100%|██████████| 938/938 [00:20<00:00, 45.38it/s, train_loss=0.327]


Epoch 5 Summary | Train Loss: 0.3251 | Val Loss : 0.3625 | Val Acc : 87.25%

 Finished Training!!! 


In [ ]:
# torch.max(a,b,c): 최대값 찾을때 사용
x = torch.tensor([
    [1,2,3],
    [4,5,6]
])
x 
# 1.max(input) => return tensor(value)
print(torch.max(x))
print(torch.max(x).item())

# 1.max(input,dim) # NamedTuple(values,index)

tensor(6)
6


In [ ]:
# 1.max(input,dim) # NamedTuple(values,index)

x = torch.tensor([
    [10,2 ,35], # 1행
    [4 ,50,6], # 2행
    [7 ,8, 9]   # 3행
])
values,indices = torch.max(x,dim=0) # 열끼리 비교
print(values,indices)
values,indices = torch.max(x,dim=1) # 행끼리 비교
print(values,indices)


# 각 열마다 

tensor([10, 50, 35]) tensor([0, 1, 0])
tensor([35, 50,  9]) tensor([2, 1, 2])
